In [1]:
workspace()

In [3]:
using Distributions: pdf, rand, Uniform, Normal

# FUNCTIONS
The following are functions for Hamiltonian Monte Carlo (HMC) and Stochastic Gradient HMC.

In [4]:
include(joinpath(homedir(), "Dropbox/MS THESIS/JULIA/HMC.jl"));
include(joinpath(homedir(), "Dropbox/MS THESIS/JULIA/SG HMC.jl"));

In [5]:
potential(x::AbstractArray{Float64}; μ::AbstractArray{Float64} = [10.; 10], Σ::AbstractArray{Float64} = eye(2)) = (x - μ)' * (Σ)^(-1) * (x - μ);
dpotential(x::AbstractArray{Float64}; μ::AbstractArray{Float64} = [10.; 10], Σ::AbstractArray{Float64} = eye(2)) = (Σ)^(-1) * (x - μ);
kinetic(p::AbstractArray{Float64}; Σ::Array{Float64} = eye(length(p))) = (p' * inv(Σ) * p) / 2;
dkinetic(p::AbstractArray{Float64}; Σ::Array{Float64} = eye(length(p))) = inv(Σ) * p;

HAMILTONIAN MONTE CARLO

In [6]:
HMC_object = HMC(potential, kinetic, dpotential, dkinetic, [0; 0], 2);
@time weights = hmc(HMC_object);

  2.780639 seconds (3.50 M allocations: 164.595 MB, 1.71% gc time)


In [7]:
mapslices(mean, weights, [1])

1×2 Array{Float64,2}:
 9.94559  9.51106

STOCHASTIC GRADIENT HAMILTONIAN MONTE CARLO

In [8]:
SGHMC_object = SGHMC(dpotential, dkinetic, eye(2), eye(2), eye(2), [0; 0], 2);
@time weights1 = sghmc(SGHMC_object);

  1.444521 seconds (2.16 M allocations: 110.469 MB, 1.76% gc time)


In [9]:
# Plot the Samples
mapslices(mean, weights1, [1])

1×2 Array{Float64,2}:
 9.88426  9.82768

In [18]:
# Plot the Samples
mapslices(mean, weights1, [1])

1×2 Array{Float64,2}:
 9.88426  9.82768

II. BAYESIAN LINEAR REGRESSION

In [64]:
# The log likelihood function is given by the following codes:
function loglike(theta::Array{Float64}; alpha::Float64 = alpha, x::Array{Float64} = x, y::Array{Float64} = y)
  yhat = theta[1] + theta[2]*x
"""
STOCHASTIC GRADIENT HAMILTONIAN MONTE CARLO

The following codes explore the use of SGHMC for Bayesian Inference
"""
immutable SGHMC
  dU      ::Function
  dK      ::Function
  dKΣ     ::Array{Float64}
  C       ::Array{Float64}
  V       ::Array{Float64}
  init_est::Array{Float64}
  d       ::Int64
end

function sghmc(Parameters::SGHMC;
  leapfrog_params::Dict{Symbol, Real} = Dict([:ɛ => .05, :τ => 20]),
  set_seed::Int64 = 123,
  r::Int64 = 1000)

  dU, dK, dKΣ, C, V, w, d = Parameters.dU, Parameters.dK, Parameters.dKΣ, Parameters.C, Parameters.V, Parameters.init_est, Parameters.d
  ɛ, τ = leapfrog_params[:ɛ], leapfrog_params[:τ]

  if typeof(set_seed) == Int64
    srand(set_seed)
  end

  x = zeros(r, d);
  B = .5 * V * ɛ
  D = sqrt(2 * (C - B) * ɛ)
  if size(B) != size(C)
    error("C and V should have the same dimension.")
  else
    if sum(size(B)) > 1
      if det(B) > det(C)
        error("ɛ is too big. Consider decreasing it.")
      end
    else
      if det(B[1]) > det(C[1])
        error("ɛ is too big. Consider decreasing it.")
      end
    end
  end

  for i in 1:r
    p = randn(d, 1)

    for j in 1:τ
      p = p - dU(w) * ɛ - C * inv(dKΣ) * p + D * randn(d, 1);
      w = w + dK(p) * ɛ;
    end

    x[i, :] = w
  end

  return x
end


  likhood = Float64[]
  for i in 1:length(yhat)
    push!(likhood, pdf(Normal(yhat[i], alpha), y[i]))
  end

  return likhood |> sum
end

# Define the log prior and lo posterior
function logprior(theta::Array{Float64}; mu::Array{Float64} = mu, s::Array{Float64} = s)
  w0_prior = log(pdf(Normal(mu[1, 1], s[1, 1]), theta[1]))
  w1_prior = log(pdf(Normal(mu[2, 1], s[2, 2]), theta[2]))
   w_prior = [w0_prior w1_prior]

  return w_prior |> sum
end

function logpost(theta::Array{Float64})
  loglike(theta, alpha = alpha, x = x, y = y) + logprior(theta, mu = mu, s = s)
end

U(theta::Array{Float64}) = - logpost(theta)
K(p::Array{Float64}; Σ = eye(length(p))) = (p' * inv(Σ) * p) / 2
function dPotential(theta::Array{Float64}; alpha::Float64 = 1/5., b::Float64 = 2.)
  [-alpha * sum(y - (theta[1] + theta[2] * x));
   -alpha * sum((y - (theta[1] + theta[2] * x)) .* x)] + b * theta
end
function dPotential_noise(theta::Array{Float64}; alpha::Float64 = 1/5., b::Float64 = 2.)
  [-alpha * sum(y - (theta[1] + theta[2] * x));
   -alpha * sum((y - (theta[1] + theta[2] * x)) .* x)] + b * theta + randn(2,1)
end
dKinetic(p::AbstractArray{Float64}; Σ::Array{Float64} = eye(length(p))) = inv(Σ) * p;

In [65]:
srand(123);
w0 = .2; w1 = -.9; stdev = 5.;

# Define data parameters
alpha = 1 / stdev; # for likelihood

# Generate Hypothetical Data
n = 5000;
x = rand(Uniform(-1, 1), n);
A = [ones(length(x)) x];
B = [w0; w1];
f = A * B;
y = f + rand(Normal(0, alpha), n);

# Define Hyperparameters
Imat = diagm(ones(2), 0);
b = 2.; # for prior
b1 = (1 / b)^2; # Square this since in Julia, rnorm uses standard dev

mu = zeros(2); # for prior
s = b1 * Imat; # for prior

2×2 Array{Float64,2}:
 0.25  0.0 
 0.0   0.25

HAMILTONIAN MONTE CARLO

In [66]:
HMC_object2 = HMC(U, K, dPotential, dKinetic, zeros(2, 1), 2);

# Sample
@time weights = hmc(HMC_object2, leapfrog_params = Dict([:ɛ => .009, :τ => 20]), r = 10000);

 72.868762 seconds (21.91 M allocations: 109.093 GB, 22.47% gc time)


In [67]:
# Plot the Samples
mapslices(mean, weights, [1])

1×2 Array{Float64,2}:
 0.202044  -0.889579

STOCHASTIC HAMILTONIAN MONTE CARLO

In [68]:
SGHMC_object = SGHMC(dPotential_noise, dKinetic, eye(2), eye(2), eye(2), [0; 0], 2.);

@time weights1 = sghmc(SGHMC_object, leapfrog_params = Dict([:ɛ => .009, :τ => 20]), r = 10000);

 35.187967 seconds (19.45 M allocations: 53.039 GB, 19.85% gc time)


In [26]:
mapslices(mean, weights1[(!isnan(weights1[:, 1]) |> find), :], [1])

[0.203508 -0.892442]